# 02: DGAN

The implementation of the DGAN comes from the [Gretel-synthetics-Repository](https://github.com/gretelai/gretel-synthetics)

In [10]:
import os

import numpy as np
import pandas as pd

from synthesizers.dgan.dgan import DGAN
from synthesizers.dgan.config import DGANConfig, OutputType


In [12]:
windows = np.load('data/wesad/wesad_windows.npy')
labels = np.load('data/wesad/wesad_labels.npy')

In [13]:
config = DGANConfig(
    max_sequence_len=windows.shape[1],
    sample_len=60,
    batch_size=min(1000, windows.shape[0]),
    apply_feature_scaling=False,
    apply_example_scaling=False,
    use_attribute_discriminator=False,
    generator_learning_rate=1e-4,
    discriminator_learning_rate=1e-4,
    epochs=10,
    cuda=True,
    #mps=True
)

print(config.batch_size)

# Train DGAN model
model = DGAN(config)

1000


In [14]:
model.train_numpy(
    windows,
    feature_types=[OutputType.CONTINUOUS] * windows.shape[2]
)
os.system("say 'DGAN IST FERTIG'")

The following device is chosen: cpu


[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x15773c700>
Traceback (most recent call last):
  File "/Users/nils/miniconda3/envs/final_requirements/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/Users/nils/miniconda3/envs/final_requirements/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/Users/nils/miniconda3/envs/final_requirements/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid 

0

In [181]:
model.save("../models/10_subs_1hz")

In [261]:
num_syn_subjects_list = [1] + [i for i in range(10, 101, 10)] + [i for i in range(200, 1001, 100)] 

In [11]:
# Generate synthetic subject
for num_syn_subjects in num_syn_subjects_list:
    num_syn_samples = len(df) // 60 // 16 * num_syn_subjects
    _, gen_data = model.generate_numpy(num_syn_samples)
    gen_data[:, :, 6] = np.where(gen_data[:, :, 6] >= 0.5, 1, 0)
    with open (f"/Users/nils/thesis/Data_Generation/data/syn/dgan/no_dp/aug/syn_subject_{num_syn_subjects}.npy", "wb") as f:
       np.save(f, gen_data)

NameError: name 'num_syn_subjects_list' is not defined

In [16]:
# Generate synthetic dataset
num_syn_samples = windows.shape[0]
_, gen_data = model.generate_numpy(num_syn_samples)
gen_data[:, :, 6] = np.where(gen_data[:, :, 6] >= 0.5, 1, 0)
with open (f"/Users/nils/thesis/Data_Generation/data/syn/dgan/no_dp/syn_dataset_{num_syn_samples}_10_subjs.npy", "wb") as f:
   np.save(f, gen_data)

In [154]:
# Compare (non-temporal) correlations between the 4 temperatures
syn_df = pd.DataFrame(gen_data.reshape(-1, gen_data.shape[2]), columns=df.columns)

print("Correlation in real data:")
print(df.corr())
print()
print("Correlation in synthetic data:")
print(syn_df.corr())


Correlation in real data:
            BVP       EDA     ACC_x     ACC_y     ACC_z      TEMP     label
BVP    1.000000  0.053760 -0.155608  0.014169  0.098046  0.019357 -0.003817
EDA    0.053760  1.000000  0.060626 -0.131111 -0.102899 -0.463535  0.813840
ACC_x -0.155608  0.060626  1.000000 -0.094749 -0.117968 -0.096538  0.026616
ACC_y  0.014169 -0.131111 -0.094749  1.000000  0.089595  0.120699 -0.052823
ACC_z  0.098046 -0.102899 -0.117968  0.089595  1.000000 -0.077563 -0.247918
TEMP   0.019357 -0.463535 -0.096538  0.120699 -0.077563  1.000000 -0.496042
label -0.003817  0.813840  0.026616 -0.052823 -0.247918 -0.496042  1.000000

Correlation in synthetic data:
            BVP       EDA     ACC_x     ACC_y     ACC_z      TEMP     label
BVP    1.000000  0.223741 -0.047836 -0.376649  0.074046  0.100492  0.195797
EDA    0.223741  1.000000 -0.297961 -0.047625 -0.215089 -0.366427  0.785659
ACC_x -0.047836 -0.297961  1.000000  0.221981  0.155055 -0.020300 -0.113978
ACC_y -0.376649 -0.047625  0.2

In [47]:
syn_df = pd.read_csv("/Users/nils/thesis/gretel-synthetics/src/syn_df_33152_sorted_new_1hz.csv", index_col=0)

In [187]:
# Generate synthetic dataset
num_syn_samples = windows.shape[0]
_, gen_data = model.generate_numpy(num_syn_samples)
gen_data[:, :, 6] = np.where(gen_data[:, :, 6] >= 0.5, 1, 0)

syn_df = pd.DataFrame(gen_data.reshape(-1, gen_data.shape[2]), columns=df.columns)
syn_df = syn_df.sort_values(by=['label'])
syn_df.reset_index(drop=True, inplace=True)
windows, labels = sliding_windows(syn_df)

syn_df.to_csv(f"/Users/nils/thesis/Data_Generation/data/syn/dgan/no_dp/syn_dataset_{num_syn_samples}_10_subjs.csv")
with open (f"/Users/nils/thesis/Data_Generation/data/syn/dgan/no_dp/syn_dataset_{num_syn_samples}_10_subjs.npy", "wb") as f:
   np.save(f, windows)